La primera linea es para definir el root del proyecto para poder ver las carpetas del proyecto desde /test
*Levanto un csv de dominios y los busco en el schema de strix y lo persisto en el schema public en postgres. Solo los domain no existentes.*
*Tambien expone metodos para persistir de un solo dominio*

Hay que probar:
1) create_domains_from_vehicles
2) borrar create_domains_from_vehicles_borrarGustavo
        

In [1]:

# 🔧 Setup del proyecto
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

print("📁 Proyecto raíz:", project_root)

📁 Proyecto raíz: c:\Desarrollo\strix-system


In [3]:
from core.infraestructure.db.postgres import Postgres

# Instancia de la conexión
db = Postgres()

# Prueba la conexión
engine = db.connPostgres()

# Valida el resultado
if engine:
    print("✅ Conexión exitosa a PostgreSQL.")
else:
    print("❌ Error al conectar con PostgreSQL.")

✅ Nueva Conexión a PostgreSQL establecida.
✅ Conexión exitosa a PostgreSQL.


In [4]:
from sqlalchemy import text

query = f"""
   SELECT *
FROM strix.vvehicle 
-- inner join public."domain" d on d.id_thing = strix.vvehicle.id
WHERE 
  (
    (LTRIM(make) ILIKE '%CHEVROLET%' AND (LTRIM(model) ILIKE '%ONIX%' ))
    OR
    (LTRIM(make) ILIKE '%TOYOTA%' AND LTRIM(model) ILIKE '%COROLLA CROS%')
  )
  AND location_datetime IS NOT NULL
  AND location_datetime > '2025-04-01 00:01:00.000 -03'
  AND strix.vvehicle.created_datetime > '2025-01-01 00:01:00.000 -03';
"""


with engine.connect() as conn:
    result = conn.execute(text(query))
    print(result.fetchone())
    if result is not None:
        #df_results.to_csv("resultados2.csv", index=False)
        print("✅ Consulta ejecutada con éxito.")
    else:
        print("❌ No se obtuvieron resultados.")

('mrn:thing:vehicle:b7dc9e61-5f52-46f0-b4c1-ee774fd313ee', 'mrn:account:6327823c-288a-4a05-921f-12ac8c716ae4', 'TOYOTA', 2023, 'GRIS PLATA', 'AF853XV', 'COROLLA CROSS 1.8 SEG HEV E-CVT', 'AF853XV', 'car', '2ZR2R91032', '9BRKAZZG4P0640828', 0, False, 339, 44060, -34.611822, -58.460513, False, False, 'mrn:thing:gps:quec001-862096071351964', datetime.datetime(2025, 6, 5, 20, 18, 2, tzinfo=datetime.timezone.utc), datetime.datetime(2025, 5, 16, 15, 46, 58, tzinfo=datetime.timezone.utc))
✅ Consulta ejecutada con éxito.


In [5]:
import pandas as pd

# Convertir el CursorResult a lista de diccionarios
rows = result.mappings().all()

# Convertir a DataFrame
df = pd.DataFrame(rows)

In [6]:

from core.services.vehicle_service import VehicleService
from core.infraestructure.uow.sqlalchemy_uow import SQLAlchemyUnitOfWork

from sqlalchemy.exc import SQLAlchemyError


try:
    domain_list = df['domain'].tolist()
    print(f"🔎 Buscando {len(domain_list)} dominios en strix.vehicle...")
    with SQLAlchemyUnitOfWork() as uow:
        VehicleService(uow).create_domains_from_vehicles(domains=domain_list)
except SQLAlchemyError as e:
    print(f"❌ Error al crear dominios desde vehicle: {e}")
    


🔎 Buscando 1185 dominios en strix.vehicle...
🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
⚠️ El dominio 'POD184' con id_account 'mrn:account:28b9e069-0ca7-45ef-828f-ed7ad328f423' ya existe. No se creará.
⚠️ El dominio 'AG917IB' con id_account 'mrn:account:f69b9d63-5e35-4534-b7e9-1d4aa6569e99' ya existe. No se creará.
⚠️ El dominio 'AE613JC' con id_account 'mrn:account:2b170cbd-7d1f-44e3-9b47-67d29d507f5a' ya existe. No se creará.
⚠️ El dominio 'AH031DU' con id_account 'mrn:account:192e33f1-3ccd-4b58-b6a9-831125aa05df' ya existe. No se creará.
⚠️ El dominio 'AE212TI' con id_account 'mrn:account:01bb6b11-2728-40ed-99b9-db0027089086' ya existe. No se creará.
⚠️ El dominio 'AG905TO' con id_account 'mrn:account:75f3c35f-d6d5-4003-ab48-c11ce279713b' ya existe. No se creará.
⚠️ El dominio 'AH107NA' con id_account 'mrn:account:aeb21ef1-8a0a-41c1-bddc-7968c6a8d810' ya existe. No se creará.
⚠️ El dominio 'AF813RX' con id_account 'mrn:account:a39

In [ ]:
# Crear la consulta SQL
query = f"""
    select * from strix.vvehicle where LTRIM(make) like '%TOYOTA%' and LTRIM(model) like '%IL%' and location_datetime is not null and  location_datetime > '2025-03-15 00:01:00.000 -0300' and created_datetime > '2025-01-01 00:01:00.000 -0300'
    order by created_datetime desc
"""

query

db = Postgres()
db.connPostgres()
# Ejecutar la consulta y guardar los resultados
df_results = db.query_to_dataframe(query)
if df_results is not None:
    #df_results.to_csv("resultados2.csv", index=False)
    print("✅ Consulta ejecutada con éxito. Resultados guardados en 'resultados.csv'")
else:
    print("❌ No se obtuvieron resultados.")

TypeError: sqlalchemy.cyextension.immutabledict.immutabledict is not a sequence

In [8]:
import pandas as pd

file_path = r"C:\Desarrollo\core\test\input\1dominios_vvehicle.xlsx"

try:
    df_domains = pd.read_excel(file_path)
    print("📄 Data cargada correctamente:")
    print(df_domains.head())  # Muestra las primeras filas para verificar
except Exception as e:
    print(f"❌ Error al leer el archivo: {e}")


📄 Data cargada correctamente:
      dominio
0  RERL883294


In [9]:
df_domains["dominio"] = df_domains["dominio"].astype(str).str.strip()
domain_list = df_domains["dominio"].tolist()
domain_list

['RERL883294']

In [10]:
# Crear la consulta SQL
query = f"""
    SELECT * FROM strix.vvehicle
    WHERE "domain" IN ({', '.join(f'\'{d}\'' for d in domain_list)})
"""

query

db = Postgres()
db.connPostgres()
# Ejecutar la consulta y guardar los resultados
df_results = db.query_to_dataframe(query)
if df_results is not None:
    #df_results.to_csv("resultados2.csv", index=False)
    print("✅ Consulta ejecutada con éxito. Resultados guardados en 'resultados.csv'")
else:
    print("❌ No se obtuvieron resultados.")

✅ Conexión a PostgreSQL establecida.
✅ Consulta ejecutada con éxito. Resultados guardados en 'resultados.csv'


In [11]:
df_results['domain']

0    RERL883294
Name: domain, dtype: object

VEHICLES POSTGRES STRIX

In [18]:
from data_access.vehicle_repository import VehicleRepository

repo = VehicleRepository()
vehicles = repo.get_vehicle_data(limit=5)
print(vehicles)

ModuleNotFoundError: No module named 'data_access'

In [13]:
# Probar con una lista de domains específica
domain_list = ["AD959HD", "AB042RZ"]
domain_list = ["RERL883294"]

domain_list = df_results['domain'].tolist()
print(f"🔹 Recuperando vehículos para domains: {domain_list}...")
filtered_vehicles = repo.get_vehicle_data(domains=domain_list, limit=10)
print(filtered_vehicles)


🔹 Recuperando vehículos para domains: ['RERL883294']...
[{'id_thing': 'mrn:thing:vehicle:a9ed6d85-eca4-4489-a3c3-3043118f603d', 'account_id': 'mrn:account:03a0c28a-21eb-4a93-a6fc-07bce890b0a2', 'domain': 'RERL883294'}]


In [14]:
from services.vehicle_service import VehicleService
vehicle_service = VehicleService()
#domain_list = ["NFR126", "DHK723", "NMU644"]
domain_list = df_results['domain'].tolist()
#result = vehicle_service.create_domains_from_vehicles(domains=domain_list, limit=10)
result = vehicle_service.create_domains_from_vehicles(domains=domain_list)

print(result)


✅ Conexión a PostgreSQL establecida.
✅ Conexión a PostgreSQL establecida.
✅ Conexión a PostgreSQL establecida.
✅ Conexión a PostgreSQL establecida.
✅ Conexión a PostgreSQL establecida.
✅ Conexión a PostgreSQL establecida.
✅ Conexión a PostgreSQL establecida.
✅ Conexión a PostgreSQL establecida.
⚠️ El dominio 'RERL883294' con id_account 'mrn:account:03a0c28a-21eb-4a93-a6fc-07bce890b0a2' ya existe. No se creará.
{'success': True, 'created_domains': []}
